# `mumble` creates input transformers for making documents.

![](https://media.giphy.com/media/3oz8xAcfZkJ0kkntoQ/giphy.gif)

In [1]:
    choices = list()
    show = True
    format = True

# `mumble` is fucking weird

Use type annotations to define a criteria for a transformer.  Place the return source code in the docstring.  Told you!

In [2]:
    def yaml(str: lambda x: x.strip().startswith('---')):
        """locals().update(dict(
            sum((
                list(dict.items()) for dict in __import__('yaml').safe_load_all('''{}''')
                ), [])))"""

In [3]:
    def graphviz(str: lambda x: x.strip().startswith('graph') or x.strip().startswith('digraph')):
        """__import__('IPython').display.display(
            __import__('graphviz').Source('''{}'''))"""

In [4]:
    def iframe(str: lambda x: x.strip().startswith('http')):
        """__import__('IPython').display.display(
            __import__('IPython').display.IFrame(
            \'\'\'{}\'\'\', 800, 600))"""

In [5]:
    def doctest(str: lambda x: x.strip().startswith('>>>')):
        """print(__import__('doctest').DocTestRunner().run(__import__('doctest').DocTest(
            [example for example in __import__('doctest').DocTestParser().parse(\'\'\'{0}\'\'\') if not isinstance(example, str)],
            __import__(__name__).__dict__, '<mumble>', None, None, \'\'\'{0}\'\'\'
        )))"""

In [6]:
    choices = yaml, graphviz, iframe, doctest

In [7]:
    from nbconvert.filters.markdown_mistune import IPythonRenderer, MarkdownWithMath
    
    class Markdown(MarkdownWithMath):
        """A mistune.Markdown object that accumulates the source code in the markdown body."""
        def render(Markdown, text):
            Markdown.renderer.source = """"""
            super().render(text)
            return Markdown.renderer.source

    class Renderer(IPythonRenderer):
        def __init__(Renderer, *args, **kwargs): 
            Renderer.source = super().__init__(*args, **kwargs) or """"""

        def block_code(Renderer, str, lang=None):
            Renderer.source += str + '\n'
            return super().block_code(str, lang=lang)

In [8]:
    from IPython.core.inputtransformer import InputTransformer
    from IPython.core.inputsplitter import IPythonInputSplitter
    from collections import UserList
    
    class Transformer(UserList, InputTransformer):
        push = UserList.append
        def reset(Transformer): 
            from inspect import getfullargspec
            from IPython import display, get_ipython
            global format, env
            
            source, Transformer.data = '\n'.join(Transformer), []

            if format: 
                """The templating business happens here."""
                if not 'env' in globals():
                    """create the environment once."""
                    from nbconvert.exporters.templateexporter import TemplateExporter
                    env = TemplateExporter().environment
                    
                source = env.from_string(source).render(**get_ipython().user_ns)
                
            if show and source and source.splitlines()[0].strip(): 
                display.display(display.Markdown(source))
                
            source = Markdown(Renderer()).render(source)
            
            for callable in choices:
                spec = getfullargspec(callable)
                if spec.annotations[spec.args[0]](source):
                    source = callable.__doc__.format(source)
                    break
            
            return _NATIVE_TRANSFORM.transform_cell(source)
        
        def register(Transformer, object): TRANSFORMERS += object,

In [9]:
    def load_ipython_extension(ip=None):
        from IPython import get_ipython
        global _NATIVE_TRANSFORM
        ip = ip or get_ipython()
        _NATIVE_TRANSFORM = globals().get('_NATIVE_TRANSFORM', ip.input_transformer_manager)
        ip.input_transformer_manager = IPythonInputSplitter(True, [], [], [Transformer()])
    
    def unload_ipython_extension(ip=None):
        from IPython import get_ipython
        global _NATIVE_TRANSFORM
        ip = ip or get_ipython()
        ip.input_transformer_manager = globals().get('_NATIVE_TRANSFORM', ip.input_transformer_manager)

    load_ipython_extension()

# This is yaml
    
    ---
    a: 10

    assert 'a' in locals()

A literate graph builder 
    
    graph graphname {
         a[href="http://google.com" target="_blank" label="Sugar"] ;
        a -- b -- c;
         b -- d;

    }

In [10]:
    if __name__ == '__main__':
        !source activate p6 && pytest tests/test_mumble.ipynb

============================= test session starts ==============================
platform darwin -- Python 3.6.3, pytest-3.2.1, py-1.4.34, pluggy-0.4.0
rootdir: /Users/tonyfast/XXX, inifile:
plugins: smartcov-0.3, ipynb-1.1.0, hypothesis-3.44.6
collected 2 items                                                               

tests/test_mumble.ipynb ..

=========================== 2 passed in 4.08 seconds ===========================
